In [1]:
abstract type Instr end

In [2]:
function run!(instr::Instr, s::Vector)::Vector end

run! (generic function with 1 method)

In [3]:
struct Seq <: Instr
    instr1::Instr
    instr2::Instr
end

In [4]:
function run!(i::Seq, s::Vector)
    run!(i.instr1, s)
    run!(i.instr2, s)
end

run! (generic function with 2 methods)

In [5]:
struct Push{T} <: Instr
    val::T
end

In [6]:
function run!(i::Push{T}, s::Vector) where {T}
    push!(s, i.val)
end

run! (generic function with 3 methods)

In [7]:
struct Add <: Instr end

In [8]:
function run!(::Add, s::Vector)
    x1 = pop!(s)
    x2 = pop!(s)
    push!(s, x2 + x1)
end

run! (generic function with 4 methods)

In [9]:
run!(Add(), Int[10, 20])

1-element Vector{Int64}:
 30

In [10]:
run!(Push(20), Int[10])

2-element Vector{Int64}:
 10
 20

In [11]:
run!(Seq(Push(10), Push(20)), Int[])

2-element Vector{Int64}:
 10
 20

In [12]:
function run!(iseq::Vector{<:Instr}, s::Array)
    for i in iseq
        run!(i, s)
    end
    s
end

run! (generic function with 5 methods)

In [13]:
run!(Instr[Push(10), Push(20), Add()], Int[])

1-element Vector{Int64}:
 30

In [14]:
struct Op2 <: Instr
    f::Function
end

In [15]:
function run!(op2::Op2, s::Vector)
    x1 = pop!(s)
    x2 = pop!(s)
    push!(s, op2.f(x2, x1))
end

run! (generic function with 6 methods)

In [16]:
run!(Instr[Push("Aa"), Push("Bb"), Op2((*))], String[])

1-element Vector{String}:
 "AaBb"

In [17]:
run!(Seq(Push(10), Seq(Push(20), Op2((+)))), Int[])

1-element Vector{Int64}:
 30

In [18]:
@code_typed run!(Instr[Push(10), Push(20), Add()], Int[])

CodeInfo(
1 ── %1  = Base.bitcast(UInt64, 1)::UInt64
│    %2  = Base.sub_int(%1, 0x0000000000000001)::UInt64
│    %3  = Base.arraylen(iseq)::Int64
│    %4  = Base.sle_int(0, %3)::Bool
│    %5  = Base.bitcast(UInt64, %3)::UInt64
│    %6  = Base.ult_int(%2, %5)::Bool
│    %7  = Base.and_int(%4, %6)::Bool
└───       goto #3 if not %7
2 ── %9  = Base.arrayref(false, iseq, 1)::Instr
│    %10 = Base.add_int(1, 1)::Int64
└───       goto #4
3 ──       goto #4
4 ┄─ %13 = φ (#2 => false, #3 => true)::Bool
│    %14 = φ (#2 => %9)::Instr
│    %15 = φ (#2 => %10)::Int64
└───       goto #5
5 ── %17 = Base.not_int(%13)::Bool
└───       goto #11 if not %17
6 ┄─ %19 = φ (#5 => %14, #10 => %34)::Instr
│    %20 = φ (#5 => %15, #10 => %35)::Int64
│          Main.run!(%19, s)::Any
│    %22 = Base.bitcast(UInt64, %20)::UInt64
│    %23 = Base.sub_int(%22, 0x0000000000000001)::UInt64
│    %24 = Base.arraylen(iseq)::Int64
│    %25 = Base.sle_int(0, %24)::Bool
│    %26 = Base.bitcast(UInt64, %24)::UInt64
│    %